In [1]:
#constans and paths
DATA_PATH='../data/raw/survey_results_public.csv'

In [2]:
import pandas as pd
import numpy as np
pd.options.display.max_rows = 10000 #?

In [3]:
#reaad data
df= pd.read_csv(DATA_PATH)

In [4]:
#print shape/dimension of the data
df.shape

(83439, 48)

In [5]:
#show first 5 entries
df.head()

,ResponseId,MainBranch,Employment,Country,US_State,UK_Country,EdLevel,Age1stCode,LearnCode,YearsCode,...,Age,Gender,Trans,Sexuality,Ethnicity,Accessibility,MentalHealth,SurveyLength,SurveyEase,ConvertedCompYearly
0,1,I am a developer by profession,"Independent contractor, freelancer, or self-em...",Slovakia,NaN,NaN,"Secondary school (e.g. American high school, G...",18 - 24 years,Coding Bootcamp;Other online resources (ex: vi...,NaN,...,25-34 years old,Man,No,Straight / Heterosexual,White or of European descent,None of the above,None of the above,Appropriate in length,Easy,62268.0
1,2,I am a student who is learning to code,"Student, full-time",Netherlands,NaN,NaN,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",11 - 17 years,"Other online resources (ex: videos, blogs, etc...",7,...,18-24 years old,Man,No,Straight / Heterosexual,White or of European descent,None of the above,None of the above,Appropriate in length,Easy,NaN
2,3,"I am not primarily a developer, but I write co...","Student, full-time",Russian Federation,NaN,NaN,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",11 - 17 years,"Other online resources (ex: videos, blogs, etc...",NaN,...,18-24 years old,Man,No,Prefer not to say,Prefer not to say,None of the above,None of the above,Appropriate in length,Easy,NaN
3,4,I am a developer by profession,Employed full-time,Austria,NaN,NaN,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",11 - 17 years,NaN,NaN,...,35-44 years old,Man,No,Straight / Heterosexual,White or of European descent,I am deaf / hard of hearing,NaN,Appropriate in length,Neither easy nor difficult,NaN
4,5,I am a developer by profession,"Independent contractor, freelancer, or self-em...",United Kingdom of Great Britain and Northern I...,NaN,England,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",5 - 10 years,Friend or family member,17,...,25-34 years old,Man,No,NaN,White or of European descent,None of the above,NaN,Appropriate in length,Easy,NaN


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83439 entries, 0 to 83438
Data columns (total 48 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   ResponseId                    83439 non-null  int64  
 1   MainBranch                    83439 non-null  object 
 2   Employment                    83323 non-null  object 
 3   Country                       83439 non-null  object 
 4   US_State                      14920 non-null  object 
 5   UK_Country                    4418 non-null   object 
 6   EdLevel                       83126 non-null  object 
 7   Age1stCode                    83243 non-null  object 
 8   LearnCode                     82963 non-null  object 
 9   YearsCode                     81641 non-null  object 
 10  YearsCodePro                  61216 non-null  object 
 11  DevType                       66484 non-null  object 
 12  OrgSize                       60726 non-null  object 
 13  C

In [7]:
# visualize a sample ---> Display Random answers
df.sample(1).iloc[0]
#.iloc---> to visualize it verically

ResponseId                                                                  76955
MainBranch                                         I am a developer by profession
Employment                                                     Employed full-time
Country                                                  United States of America
US_State                                                                 Missouri
UK_Country                                                                    NaN
EdLevel                              Bachelor’s degree (B.A., B.S., B.Eng., etc.)
Age1stCode                                                          25 - 34 years
LearnCode                       Other online resources (ex: videos, blogs, etc...
YearsCode                                                                       5
YearsCodePro                                                                    3
DevType                                                       Developer, back-end
OrgSize         

In [8]:
df.describe()

,ResponseId,CompTotal,ConvertedCompYearly
count,83439.000000,4.718300e+04,4.684400e+04
mean,41720.000000,2.119407e+69,1.184262e+05
std,24086.908893,4.603702e+71,5.272944e+05
min,1.000000,0.000000e+00,1.000000e+00
25%,20860.500000,1.600000e+04,2.702500e+04
50%,41720.000000,6.700000e+04,5.621100e+04
75%,62579.500000,1.400000e+05,1.000000e+05
max,83439.000000,1.000000e+74,4.524131e+07


In [9]:
#columns with unpredicted types
questionable_cols=['YearsCodePro','YearsCode','Age','Age1stCode']

In [10]:
#print unique values of questionable_cols

for col in questionable_cols:
    print(col)
    print(df[col].unique().tolist())
    print("----------------------------")
    print()

YearsCodePro
[nan, '10', '4', '5', '6', '2', '30', '9', '18', '12', '21', '1', '16', 'Less than 1 year', '15', '3', '35', '7', '8', '17', '14', '26', '25', '20', '50', '34', '11', '24', '22', '13', '31', '23', '39', '41', '27', '28', '19', '33', 'More than 50 years', '37', '29', '32', '43', '40', '38', '45', '42', '46', '36', '44', '47', '48', '49']
----------------------------

YearsCode
[nan, '7', '17', '3', '4', '6', '16', '12', '15', '10', '40', '9', '26', '14', '39', '20', '8', '19', '5', 'Less than 1 year', '22', '2', '1', '34', '21', '13', '25', '24', '30', '31', '18', '38', 'More than 50 years', '27', '41', '42', '35', '23', '28', '11', '37', '44', '43', '36', '33', '45', '29', '50', '46', '32', '47', '49', '48']
----------------------------

Age
['25-34 years old', '18-24 years old', '35-44 years old', 'Prefer not to say', '45-54 years old', 'Under 18 years old', '65 years or older', '55-64 years old', nan]
----------------------------

Age1stCode
['18 - 24 years', '11 - 17 ye